# Simplified Medical Equipment Dashboard

A clean, single-notebook dashboard to visualize Medical Equipment Utilization, ROI, and EDA metrics.
Key features:
- **EDA**: Data overview and distributions.
- **Utilization**: Operating hours and utilization rates.
- **ROI**: Financial performance and profit margins.

In [ ]:
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output

# --- 1. Data Loading & Cleaning ---

file_path = "../../data/processed/medical_equipment_utilization_synthetic_cleaned(in).csv"
df = pd.read_csv(file_path)

# Clean column names
df.columns = [c.strip().replace(" ", "_") for c in df.columns]
df = df.dropna(axis=1, how="all")

# Clean currency columns
currency_cols = ["Cost_per_procedure", "Daily_Operating_Cost", "Procedure_Revenue", "Net-Profit_(daily)"]
for col in currency_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(
            df[col].astype(str).str.replace(r"[KES, ]", "", regex=True),
            errors="coerce"
        )

# Clean numeric columns
numeric_cols = ["Available_Hours", "Actual_Operating_Hours", "Utilization_Rate", "Number_of_Procedures"]
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

if "Date" in df.columns:
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

print("Data Loaded. Rows:", len(df))

In [ ]:
# --- 2. Dashboard App Setup ---

app = JupyterDash(__name__)

# Styles
app.layout = html.Div([
    html.H1("Medical Equipment Analysis Dashboard", style={'textAlign': 'center', 'color': '#2c3e50'}),
    
    dcc.Tabs(id="tabs", value="tab-eda", children=[
        dcc.Tab(label="EDA Overview", value="tab-eda"),
        dcc.Tab(label="Utilization Metrics", value="tab-util"),
        dcc.Tab(label="Financial ROI", value="tab-roi"),
    ]),
    
    html.Div(id="content", style={'padding': '20px'})
])

# --- 3. Callbacks & Visualizations ---

@app.callback(Output("content", "children"), Input("tabs", "value"))
def render_content(tab):
    if tab == "tab-eda":
        # EDA Charts
        fig1 = px.histogram(df, x="Utilization_Rate", nbins=30, title="Utilization Rate Distribution")
        fig2 = px.bar(
            df["Equipment_Type"].value_counts().reset_index(), 
            x="index", y="Equipment_Type", 
            labels={'index': 'Equipment Type', 'Equipment_Type': 'Count'},
            title="Records by Equipment Type"
        )
        return html.Div([dcc.Graph(figure=fig1), dcc.Graph(figure=fig2)])
    
    elif tab == "tab-util":
        # Utilization Charts
        avg_util = df.groupby("Equipment_Type")["Utilization_Rate"].mean().reset_index()
        fig1 = px.bar(avg_util, x="Equipment_Type", y="Utilization_Rate", title="Avg Utilization by Type")
        
        fig2 = px.scatter(
            df, x="Available_Hours", y="Actual_Operating_Hours", 
            color="Equipment_Type", title="Operating vs Available Hours"
        )
        return html.Div([dcc.Graph(figure=fig1), dcc.Graph(figure=fig2)])
    
    elif tab == "tab-roi":
        # ROI Charts
        roi_df = df.groupby("Equipment_Type")[["Net-Profit_(daily)", "Procedure_Revenue"]].sum().reset_index()
        roi_df["ROI %"] = (roi_df["Net-Profit_(daily)"] / roi_df["Procedure_Revenue"]) * 100
        
        fig1 = px.bar(roi_df, x="Equipment_Type", y="ROI %", title="ROI % by Equipment Type")
        fig2 = px.bar(roi_df, x="Equipment_Type", y="Net-Profit_(daily)", title="Total Net Profit by Type")
        
        return html.Div([dcc.Graph(figure=fig1), dcc.Graph(figure=fig2)])

# --- 4. Run Server ---
if __name__ == '__main__':
    app.run_server(mode="inline", height=800)